In [3]:
import torch
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use(["science", "notebook", "grid"])

# print versions of modules
print("torch version: ", torch.__version__)
print("pandas version: ", pd.__version__)
print("numpy version: ", np.__version__)
print("matplotlib version: ", matplotlib.__version__)
print("seaborn version: ", sns.__version__)

torch version:  1.13.1
pandas version:  1.5.2
numpy version:  1.23.4
matplotlib version:  3.6.2
seaborn version:  0.12.1


In [4]:
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    print("Metal performance shaders (MPS) are available! Let the show begin")
    mps_device = torch.device("mps")

Metal performance shaders (MPS) are available! Let the show begin


Read data from `csv` format and convert into `numpy ndarrays` after dropping the `condition` column

In [5]:
test_set = pd.read_csv("/Users/archismanchakraborti/Desktop/python files/Dissertation/filtered_data_test.csv")\
                                                .drop(columns = ["Condition"]).values
train_set = pd.read_csv("/Users/archismanchakraborti/Desktop/python files/Dissertation/filtered_data.csv")\
                                            .drop(columns = ["Condition"]).values

Converting arrays into `Torch Tensors`

## Create Dataset

In [6]:
from torch.utils.data import Dataset

In [7]:
class ECG_data(Dataset):
    def __init__(self, link):
        data = pd.read_csv(link).drop(columns = ["Condition"]).values
        self.x = torch.from_numpy(data[:, :-1]).type(torch.float32).to(mps_device) # Features
        self.y = torch.from_numpy(data[:, [-1]]).type(torch.float32).to(mps_device) # labels
        self.n_samples = data.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self) -> int:
        return self.n_samples        

In [8]:
train_link = "filtered_data.csv"
test_link = "filtered_data_test.csv"

train_set = ECG_data(link=train_link)
test_set = ECG_data(link=test_link)

## Print some relevant information

In [9]:
print(f"Length of training set = {len(train_set)}")
print(f"Length of testing set = {len(test_set)}")
print(f"No of classes to predict = 5")

Length of training set = 87554
Length of testing set = 21892
No of classes to predict = 5


## Create `Dataloader objects`

In [10]:
from torch.utils.data import DataLoader

In [11]:
train_dl = DataLoader(dataset = train_set, batch_size = 128, shuffle = True, num_workers = 0)
train_dl = DataLoader(dataset = train_set, batch_size = 128, shuffle = True, num_workers = 0)

# Iterator for dataloader
dataiter = iter(train_dl)
data = next(dataiter)

device(type='mps', index=0)